# GEOG-503 Midterm Exam

## Due date: 
Oct 19 (Friday) 9:30 am

## What to submit:

The Jupyter Notebook that contains solutions to the following questions. File name convention for submission: ** lastname_firstname_midterm.ipynb **

It must be submitted to [myCourses](https://mycourses.binghamton.edu/)

## Questions

Answer the questions or complete the tasks outlined in bold below, use the specific method described if applicable.

## Data
* http://spatial.binghamton.edu/geog503/data/midterm_data.zip

# Question 1

Write a program that repeatedly prompts a user for numbers until the user enters '**done**'. Once 'done' is entered, print out the **smallest, largest, mean, median, and sum** of the numbers. If the user enters anything other than a valid number put out an appropriate message and ignore the number. (25 pt.)


In [1]:
import numpy as np
values_list = []
while True:
    try:
        in_str = raw_input("Enter a number (enter 'done' to quit): ")
        if in_str == "done" and len(values_list) > 0:
            print("The numbers you entered are: {}".format(values_list))
            print("Smallest: {}".format(np.min(values_list)))
            print("Largest: {}".format(np.max(values_list)))
            print("Mean: {}".format(np.mean(values_list)))
            print("Median: {}".format(np.median(values_list)))
            print("Sum: {}".format(np.sum(values_list)))
            break   
        else: 
            values_list.append(float(in_str))
    except:
         print("Please enter a valid number")

Enter a number (enter 'done' to quit): 1
Enter a number (enter 'done' to quit): 3
Enter a number (enter 'done' to quit): 5
Enter a number (enter 'done' to quit): 7
Enter a number (enter 'done' to quit): 6
Enter a number (enter 'done' to quit): 2
Enter a number (enter 'done' to quit): haha
Please enter a valid number
Enter a number (enter 'done' to quit): this is fun!
Please enter a valid number
Enter a number (enter 'done' to quit): done
The numbers you entered are: [1.0, 3.0, 5.0, 7.0, 6.0, 2.0]
Smallest: 1.0
Largest: 7.0
Mean: 4.0
Median: 4.0
Sum: 24.0


# Question 2

Write a script that copies feature classes and raster datasets from **testdata.gdb** to the Results folder. Feature classes can be saved as Shapefile format while raster datasets can be saved as TIF format. The output file names must meet the following requirements:

* For a feature class, the output file name must be prefixed by its shape type: **Point/Polyline/Polygon**. For example: Polygon_buffers.shp, Point_NWIS_sites.shp.    

* For a raster dataset, the output file name must be prefixed by the word: **Raster**. For example: Raster_Landcover.tif

Note that you cannot use hard-coded filename to specify which file to copy. (25 pt.)

In [2]:
try:
    import archook #The module which locates arcgis
    archook.get_arcpy()
    import arcpy
except ImportError:
    print("import arcpy error")

In [3]:
import arcpy
import os
workspace = r"C:\geog503\data\testdata.gdb"
out_dir = r"C:\Geog503\data\Results"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True
fc_list = arcpy.ListFeatureClasses()
for fc in fc_list:
    desc = arcpy.Describe(fc)
    shapeType = desc.shapeType
    out_name = shapeType + "_" + fc + ".shp"
    out_path = os.path.join(out_dir, out_name)
    print("{:15} is a {:8} feature class... Copying to {}".format(fc, shapeType, out_path))
    arcpy.CopyFeatures_management(fc, out_path)

buffers         is a Polygon  feature class... Copying to C:\Geog503\data\Results\Polygon_buffers.shp
LittlePolys     is a Polygon  feature class... Copying to C:\Geog503\data\Results\Polygon_LittlePolys.shp
Watersheds      is a Polygon  feature class... Copying to C:\Geog503\data\Results\Polygon_Watersheds.shp
rivers          is a Polyline feature class... Copying to C:\Geog503\data\Results\Polyline_rivers.shp
weedpt          is a Point    feature class... Copying to C:\Geog503\data\Results\Point_weedpt.shp
NWIS_sites      is a Point    feature class... Copying to C:\Geog503\data\Results\Point_NWIS_sites.shp


In [4]:
ras_list = arcpy.ListRasters()
for ras in ras_list:
    desc = arcpy.Describe(ras)
    datasetType = desc.datasetType
    out_name = "Raster_" + ras + ".tif"
    out_path = os.path.join(out_dir, out_name)
    print("{:15} is a {:8}... Copying to {}".format(ras, datasetType, out_path))
    arcpy.CopyRaster_management(ras, out_path)

ElevationFloat  is a RasterDataset... Copying to C:\Geog503\data\Results\Raster_ElevationFloat.tif
ElevationInt    is a RasterDataset... Copying to C:\Geog503\data\Results\Raster_ElevationInt.tif
Geology         is a RasterDataset... Copying to C:\Geog503\data\Results\Raster_Geology.tif
Landcover       is a RasterDataset... Copying to C:\Geog503\data\Results\Raster_Landcover.tif


# Question 3

Write a script that converts the **airports.txt** into a Point-type shapefile. The resulting shapefile should have the following fields based on the information from the airports.txt, including **NAME, LOCID, X, and Y**.  (25 pt.)

In [5]:
import arcpy
import os
workspace =  r"C:\Geog503\data\Results"
infile = r"C:\Geog503\data\airports.txt"
prj =  r"C:\Geog503\data\airports.prj"

arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

pointList =[]
fc = os.path.splitext(os.path.basename(infile))[0] + ".shp"

f = open(infile)
for line in f.readlines():
    list1 = line.split(", ")
    point = []
    for item in list1:
        subitem = item.split(": ")
        point.append(subitem[1].strip('\n'))
    pointList.append(point)

spatialRef = arcpy.SpatialReference(prj)
arcpy.CreateFeatureclass_management(workspace,fc,"Point",spatial_reference=spatialRef)

arcpy.AddField_management(fc,"Name","string",field_length=50)
arcpy.AddField_management(fc,"LOCID","string",field_length=10)
arcpy.AddField_management(fc,"X","float")
arcpy.AddField_management(fc,"Y","float")

fields = ["Id","NAME","LOCID","X","Y","SHAPE@X","SHAPE@Y"]
cursor = arcpy.da.InsertCursor(fc, fields)

for point in pointList:
    id = int(point[0])
    name = point[1]
    locid = point[2]
    x = float(point[3])
    y = float(point[4])
    cursor.insertRow([id,name,locid,x,y,x,y])
del cursor
print("The output is saved at: {}".format(os.path.join(workspace, fc)))

The output is saved at: C:\Geog503\data\Results\airports.shp


# Question 4

Write a script that creates a minimum bounding rectangle for the **Hawaii.shp** feature class. There is a tool called [Minimum Bounding Geometry](https://goo.gl/PdRWUP) that can do this. However, your script should NOT use the tool arcpy.MinimumBoundingGeometry_management. Instead, use the [hullRectange](https://goo.gl/QJxpmp) properties of a Geometry object to create the minimum bounding rectange (25 pt.)

In [6]:
import arcpy
fc = r"C:\Geog503\data\hawaii.shp"
workspace = r"C:\Geog503\data\Results"
out_file = "Q4_envelope.shp"
arcpy.env.workspace = workspace
geomList = arcpy.CopyFeatures_management(fc,arcpy.Geometry())
pnt = arcpy.Point()
array = arcpy.Array()
for geom in geomList:
    coord_list = geom.hullRectangle.split(" ")
    point_count = len(coord_list)/2
    print('Coordinates of the minimum bounding rectangle:')
    for i in range(0, point_count):
        X = coord_list[i*2]
        Y =  coord_list[i*2+1]
        pnt.X = X
        pnt.Y = Y
        array.add(pnt)
        print('X = {}, Y = {}'.format(X,Y))
poly = arcpy.Polygon(array)
arcpy.CopyFeatures_management(poly,out_file)          

Coordinates of the minimum bounding rectangle:
X = 369452.444658486, Y = 2413270.8632363
X = 453956.568571401, Y = 2540498.57597544
X = 966495.579671934, Y = 2200072.27127229
X = 881991.455755093, Y = 2072844.55853016


<Result 'C:\\Geog503\\data\\Results\\Q4_envelope.shp'>